In [3]:
import pandas as pd
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

example_text = 'Hello, how are you?'
bert_input = tokenizer(example_text,padding='max_length', max_length = 10, 
                       truncation=True, return_tensors="pt")


print(bert_input['input_ids'])
print(bert_input['token_type_ids'])
print(bert_input['attention_mask'])

tensor([[  101,   146,  1209,  2824,  2508, 26173,  3568,   102,     0,     0]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 0, 0]])


In [4]:
example_text = tokenizer.decode(bert_input.input_ids[0])
print(example_text)

[CLS] I will watch Memento tonight [SEP] [PAD] [PAD]


In [5]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = df['label'].to_list()
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [6]:
df = pd.read_csv('../data/raw_data.csv')
print(df.shape)
df = df[['hatespeech','text']]
df.rename(columns={'hatespeech':'label'},inplace=True)
df.head()

(135556, 131)


,label,text
0,0,Yes indeed. She sort of reminds me of the elde...
1,0,The trans women reading this tweet right now i...
2,2,Question: These 4 broads who criticize America...
3,0,It is about time for all illegals to go back t...
4,2,For starters bend over the one in pink and kic...


In [29]:
df['label'] =df['label'].replace(2,1)

In [31]:
from sklearn.model_selection import train_test_split

df = df.sample(frac=0.1, random_state=42)

x_train_, x_test, y_train_, y_test = train_test_split(df.index.values,df.label.values,test_size=0.1,random_state=42,stratify=df.label.values)
x_train, x_val, y_train, y_val = train_test_split(df.loc[x_train_].index.values,df.loc[x_train_].label.values,test_size=0.2,random_state=42,stratify=df.loc[x_train_].label.values)

df_train = df.loc[x_train]
df_val = df.loc[x_val]
df_test = df.loc[x_test]
len(df_train), len(df_val), len(df_test)

(9760, 2440, 1356)

In [32]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [33]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:

            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  


In [34]:
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 4880/4880 [16:21<00:00,  4.97it/s]


Epochs: 1 | Train Loss:  0.362                 | Train Accuracy:  0.718                 | Val Loss:  0.237                 | Val Accuracy:  0.800


100%|██████████| 4880/4880 [16:22<00:00,  4.97it/s]


Epochs: 2 | Train Loss:  0.220                 | Train Accuracy:  0.815                 | Val Loss:  0.225                 | Val Accuracy:  0.801


100%|██████████| 4880/4880 [16:22<00:00,  4.97it/s]


Epochs: 3 | Train Loss:  0.182                 | Train Accuracy:  0.854                 | Val Loss:  0.237                 | Val Accuracy:  0.803


100%|██████████| 4880/4880 [16:22<00:00,  4.97it/s]


Epochs: 4 | Train Loss:  0.145                 | Train Accuracy:  0.896                 | Val Loss:  0.238                 | Val Accuracy:  0.809


100%|██████████| 4880/4880 [16:22<00:00,  4.97it/s]


Epochs: 5 | Train Loss:  0.112                 | Train Accuracy:  0.925                 | Val Loss:  0.263                 | Val Accuracy:  0.805


In [35]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.770


In [38]:
torch.save(model.state_dict(), '../models/model.pth')

In [40]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = BertClassifier()
load_model.load_state_dict(torch.load('../models/model.pth'))
load_model = load_model.to(device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [42]:
def classify(input_text):
  input_tokens = tokenizer(input_text, 
                          padding='max_length',
                          max_length = 512, 
                          truncation=True,
                          return_tensors="pt")
  mask = input_tokens['attention_mask'].to(device)
  input_id = input_tokens['input_ids'].squeeze(1).to(device)
  output = load_model(input_id, mask)
  pred = output.argmax(dim=1)
  return 'Hatespeech' if pred == 1 else 'Neutral'